In [14]:
from excellib import set_excel_color_monthly
from stocklib import get_stock_price, get_OTC_price
from finance import get_yearly, get_df_monthly, get_df_monthly_check, get_df_quarterly
from finance import get_this_quarter, get_quarter_income, get_quarter_finance, set_growth_eps
from finance import get_df_quarterly_all, get_df_yearly_all

import pandas as pd
import numpy as np
import time
from tqdm import tqdm
from datetime import datetime, timedelta
import os

root = 'D:/Mia/Stock/'
path_quarterly = root + 'quarterly/'
path_quarterly_month = path_quarterly + str(datetime.now().month).zfill(2)
path_monthly = root + 'monthly/'
path_month_finance = root + 'monthly_finance/'
path_yearly = root + 'yearly/'
path_finance = root + 'finance/'
path_trust = root + 'trust/'
folder_name = 'trust/'
s_today = datetime.now().strftime('%Y_%m_%d')
s_today_hm = datetime.now().strftime('%Y_%m_%d_%H%M')
print(s_today, s_today_hm)

if not os.path.exists(path_quarterly_month):
    print('create ', path_quarterly_month)
    os.makedirs(path_quarterly_month)

2022_03_30 2022_03_30_2130 D:/Mia/Stock/quarterly/03
create  D:/Mia/Stock/quarterly/03


In [3]:
# 公司發行股數
path_company_stock_num = root + 'company_list.xlsx'

# # 取得今日股價
today = datetime.today().date() #- timedelta(days=5)
this_year = today.year
this_quarter = get_this_quarter()

n = 1
if today.weekday() == 0:
    n = 3

print('今天:', today, '第', this_quarter, '季')
# 台灣證券交易所
df_today = get_stock_price(today)

今天: 2022-03-30 第 1 季
取得今日上市股票資訊: 20220330


In [6]:
# # 月報
# #### 彙總報表 (單位：千元)
# https://mops.twse.com.tw/mops/web/t21sc04_ifrs

# # 從月報 篩選出公司
# 上個月營收 > 30億
# 上個月營收成長 > 20% or 去年同季營收成長 > 20%
df_monthly_check = get_df_monthly_check(path_month_finance, 5, 10, 10, True)
df_monthly_check.head(1)

從月報 篩選出幾家公司
上個月營收 > 5億 上個月營收成長 > 20% or 去年同季營收成長 > 20%
(13429, 14)
目前月報年份： 111/1
所有公司筆數： 971
本月觀察公司筆數： 193


D:\Mia\Stock\code\finance.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_monthly_check['公司代號'] = df_monthly_check['公司代號'].astype(str)


,出表日期,資料年月,公司代號,公司名稱,產業別,營業收入-當月營收,營業收入-上月營收,營業收入-去年當月營收,營業收入-上月比較增減(%),營業收入-去年同月增減(%),累計營業收入-當月累計營收,累計營業收入-去年累計營收,累計營業收入-前期比較增減(%),備註
12475,111/03/30,111/2,1215,卜蜂,食品工業,20.49,26.73,16.52,-23.34,24.07,47.22,36.89,28.0,-


In [16]:
lst_monthly_id = list(df_monthly_check['公司代號'].unique())
print(lst_monthly_id)

['1215', '1216', '1232', '1301', '1303', '1305', '1307', '1310', '1321', '1338', '1409', '1444', '1451', '1473', '1476', '1507', '1532', '1537', '1541', '1558', '1604', '1605', '1608', '1612', '1614', '1626', '1708', '1709', '1712', '1718', '1722', '1723', '1773', '2002', '2013', '2022', '2023', '2027', '2028', '2029', '2030', '2031', '2034', '2049', '2059', '2069', '2104', '2204', '2208', '2211', '2227', '2239', '2247', '2303', '2313', '2327', '2328', '2330', '2337', '2338', '2340', '2344', '2345', '2376', '2377', '2379', '2383', '2388', '2397', '2408', '2430', '2441', '2449', '2454', '2457', '2481', '2511', '2520', '2542', '2543', '2545', '2548', '2603', '2606', '2607', '2609', '2610', '2614', '2615', '2618', '2636', '2637', '2845', '2849', '2851', '2881', '2885', '2887', '2888', '2890', '2891', '2906', '3010', '3017', '3022', '3026', '3030', '3034', '3035', '3036', '3037', '3059', '3090', '3189', '3209', '3305', '3443', '3454', '3515', '3528', '3532', '3533', '3557', '3576', '3592',

In [7]:

# ==============================
#  取得觀察中公司的季報、預測EPS
# ==============================

df_quarterly_all = get_df_quarterly_all()
df_yearly_all = get_df_yearly_all() # 寫入歷年EPS、本益比，估算今年股價min, max範圍
df_estimate = pd.DataFrame(columns = list(df_quarterly_all.columns)+['普通股數'])
# 取得公司發行股數
df_company_stock_num = pd.read_excel(path_company_stock_num, engine='openpyxl', dtype={'stock_id':str})
df_company_stock_num.set_index('stock_id', inplace=True)

print('觀察中公司有', len(lst_monthly_id), '筆')

觀察中公司有 193 筆


In [8]:
sep_num = 50
n = 0
lst_all_id = []
if len(lst_monthly_id)>sep_num:
    while n < len(lst_monthly_id):
        lst_all_id.append(lst_monthly_id[n:n+sep_num])
        n += sep_num
else:
    lst_all_id.append(lst_monthly_id)
for i in lst_all_id:
    print(len(i))

50
50
50
43


In [9]:
for j in range(1, len(lst_all_id)):
    print('Check:', j)
    lst_monthly_id = lst_all_id[j]
    
    for i in tqdm(range(len(lst_monthly_id))):
        stock_id = lst_monthly_id[i]
        df = get_df_quarterly(stock_id)
        t_wait = np.random.randint(3,10)
        print(stock_id, ', wait ', t_wait)
        if df is None:
            print('df is None')
            continue
        if df.loc[df.index[0], '毛利率(%)'] < 20:
            print(stock_id, '：毛利率', df.loc[df.index[0], '毛利率(%)'], '< 20%，跳過')
            continue
        elif df.loc[df.index[0], '營益率(%)'] < 8:
            print(stock_id, '：營益率 < 8%，跳過')
            continue

        # 計算歷年同季的營收成長率、EPS
        df = set_growth_eps(df, df_today, stock_id)

    #     將年度EPS寫到歷年excel裡
        lst_year = set(df.index.to_series().apply(lambda x:x[:4]))
        min_year = int(min(lst_year)) # 若最小年是2018 ，只能確定2019都有資料
        lst_year = [y for y in lst_year if int(y) > min_year]
        lst_EPS = [{'year':y, 'eps': df.loc[str(y)+'Q1', 'EPS(Y)']} for y in lst_year]

        df.reset_index(inplace=True)
        df = df.rename(columns={'index':'quarter'}) # reset_index後column名稱是index

        stock_name = ""
        if stock_id in df_today.index:
            stock_name = df_today.loc[stock_id, 'name']
            df_yearly = get_yearly(stock_id, lst_EPS, df['quarter'][0][:4], stock_name)    
            df_yearly_all = df_yearly_all.append(df_yearly, ignore_index=True)

        df.insert(0, 'name', value=stock_name)
        df.insert(0, 'stock_id', value=stock_id)

        df_quarterly_all = df_quarterly_all.append(df, ignore_index=True)
        time.sleep(t_wait)

        # ===============================
        # 預測EPS ( check_EPS )
        # ===============================
        df_new_quarter = get_quarter_finance(stock_id, df, this_year, this_quarter, df_company_stock_num, df_today)
        df_concat = pd.concat([df, df_new_quarter])
        df_concat.sort_values('quarter', ascending=False, inplace=True)
        df_estimate = df_estimate.append(df_concat)

    # # 季報
    s_today_hm = datetime.now().strftime('%Y_%m_%d_%H%M')
    print('本月觀察公司筆數：', len(df_quarterly_all['stock_id'].unique()), df_quarterly_all['stock_id'].unique())
    path_quarterly_each = path_quarterly_month + 'finance_quarterly_{}.xlsx'.format(s_today_hm)
    set_excel_color_monthly(df_quarterly_all, path_quarterly_each)

    # # 每年 EPS 預估股價
    path_monthly_eps = path_yearly + 'finance_yearly_{}.xlsx'.format(s_today_hm)
    set_excel_color_monthly(df_yearly_all, path_monthly_eps)

    # 預測EPS ( check_EPS )
    path_estimate = path_quarterly_month +'estimate_{}.xlsx'.format(s_today_hm)
    df_estimate.to_excel(path_estimate, index=False)
    print(df_estimate.head(2))


    # ===============================
    # # 取得公司歷年發放股利
    # lst = ['2615']
    # get_company_dividen(lst)
    # ===============================

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

Check: 1


c:\users\mia\appdata\local\programs\python\python37\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
  2%|█▋                                                                                 | 1/50 [00:05<04:05,  5.02s/it]

2227 , wait  5
2227 ：毛利率 13.2 < 20%，跳過


  4%|███▎                                                                               | 2/50 [00:07<03:02,  3.81s/it]

...

 96%|██████████████████████████████████████████████████████████████████████████████▋   | 48/50 [06:53<00:22, 11.09s/it]c:\users\mia\appdata\local\programs\python\python37\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
 98%|████████████████████████████████████████████████████████████████████████████████▎ | 49/50 [06:57<00:08,  8.94s/it]

5538 , wait  3
5538 ：毛利率 19.8 < 20%，跳過
5608 , wait  7
5608  wait  6


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [07:12<00:00,  8.65s/it]


本月觀察公司筆數： 58 ['2303' '2327' '2330' '2337' '2338' '2340' '2344' '2345' '2376' '2379'
 '2383' '2408' '2441' '2449' '2454' '2481' '2520' '2542' '2545' '2548'
 '2603' '2606' '2609' '2610' '2615' '2618' '2636' '2637' '2845' '2849'
 '2881' '2885' '2887' '2888' '2890' '2891' '3022' '3030' '3034' '3035'
 '3037' '3189' '3443' '3515' '3532' '3533' '3557' '3592' '3653' '3665'
 '4551' '4755' '4927' '4961' '5269' '5522' '5534' '5608']


FileCreateError: [Errno 13] Permission denied: 'D:/Mia/Stock/quarterly/trust/estimate_2022_03_30.xlsx'